In [1]:
import telebot
from time import time
import pandas as pd
from os import listdir
from pprint import pprint

In [2]:
token = '783192310:AAGHRbqknl-gArkYLcbTYmk2necszfIhQuA'

In [3]:
bot = telebot.TeleBot(token, parse_mode=None) # You can set parse_mode by default. HTML or MARKDOWN

In [4]:
data_path = '../data/'
# data_path = '/media/evilcube/98D60E9BD60E79B0/Users/EvilCube/Desktop/Projeto Integrador/instacart/data/' # Marcelo Linux

In [5]:
# loadings
df_produto= pd.read_csv(data_path + 'products.csv')
df_order = pd.read_csv(data_path + 'order_products__prior.csv') #apenas prior - fazer teste com a base train
df_depat = pd.read_csv(data_path + 'departments.csv')
df_pedido= pd.read_csv(data_path + 'orders.csv')
df_aisles = pd.read_csv(data_path + 'aisles.csv')

In [6]:
# Joins
df_join=pd.merge(df_order,df_produto,  
                 on='product_id', 
                 how='left')

df_join=pd.merge(df_join,df_depat ,  
                 on='department_id', 
                 how='left')

df_join=pd.merge(df_join,df_pedido ,  
                 on='order_id', 
                 how='left')

In [7]:
def get_most_ordered_products(df_base, user_id):
    return "\n".join(list(df_base[df_base["user_id"]==user_id].groupby('product_name').count().sort_values('order_id')[:15].index))

# print(get_most_ordered_products(df_join,202279))

In [8]:
def get_last_order_products(user_id, df_base):
    last_order_id =  df_base[df_base["user_id"]==user_id]['order_id'].max() ### ordenar por order_number e selecionar order_id
    last_order = df_base[(df_base['user_id']==user_id) & (df_base['order_id'] == last_order_id)]
    return last_order.groupby('product_name')['product_name'].count()
    # arrumar o print
    
# test
# get_last_order_products(202279, df_join)

# Mensagens padrão

In [9]:
@bot.message_handler(commands=['start'])
def send_welcome(message):
    chat_id = message.chat.id
    text = """
    Bem vind@ ao Instacart bot :) 🥕
    
    Digite a opção desejada:
    /1 - Ver o status do pedido
    /2 - Quero fazer um pedido
    /3 - Reagendar uma entrega
    /4 - Fazer uma reclamação
    """
    bot.send_message(chat_id, text)

@bot.message_handler(commands=['1'])
def send_welcome(message):
    chat_id = message.chat.id
    text = """
    Sua compra será entregue às 15h - 05/07
    """
    bot.send_message(chat_id, text)
    
@bot.message_handler(commands=['2'])
def send_welcome(message):
    chat_id = message.chat.id
    text = """
    /21 - Refazer o último pedido
    /22 - Fazer uma nova compra com os produtos mais solicitados
    /23 - Fazer uma nova compra
    """
    bot.send_message(chat_id, text)

@bot.message_handler(commands=['21'])
def send_welcome(message):
    chat_id = message.chat.id
    text = f"""
    Seu último pedido:
    {get_last_order_products(202279, df_join)}
    
    Gostaria de confirmar pedido?
    /sim
    /não
    """
    bot.send_message(chat_id, text)

@bot.message_handler(commands=['sim'])
def send_welcome(message):
    chat_id = message.chat.id
    text = f"""
    Pedido confirmado! 
    """
    bot.send_message(chat_id, text)

# Mensagens churn

In [ ]:
def churn_message(bot, user_id, name, days):
    text = f'''
Olá {name}! Como você está?

Verificamos que sua última compra foi há mais de {days} dias, e para te ajudar posso montar seu carrinho.
E nos próximos dois dias a taxa de entrega é por nossa conta :)

Deseja ver as sugestões que eu preparei especialmente pra você?

/ultimo_pedido
/mais_comprados
/nao
'''
    bot.send_message(user_id, text)

In [11]:
@bot.message_handler(commands=['mais_comprados'])
def churn_yes(message):
    chat_id = message.chat.id
    text = f'''
    O que você acha desse carrinho que montei especialmente para você ❤️:
    
    {get_most_ordered_products(df_join,202279)}
    '''
    bot.send_message(chat_id, text)

@bot.message_handler(commands=['ultimo_pedido'])
def churn_yes(message):
    chat_id = message.chat.id
    text = f'''
    O que você acha desse carrinho que montei especialmente para você ❤️:
    
    {get_last_order_products(202279, df_join)}
    '''
    bot.send_message(chat_id, text)

@bot.message_handler(commands=['nao'])
def churn_yes(message):
    chat_id = message.chat.id
    text = f'''
Que pena :(

Quando precisar de algo é só chamar com /start
    '''
    bot.send_message(chat_id, text)


#churn_message(bot, 1313926372, 'Rafaela', 10)

In [ ]:
# bot.send_message(chat_id = '@pads_instacart', text = 'Hellos')

In [ ]:
# Start "Server"
bot.polling()

In [7]:
bot.send_message(chat_id = 1313926372, text = 'Hellos')

* Pedidos mais frequentes
 - calcular a média de itens por usuário ROUND UP
* Arrumar a formatação
* perguntar (fechar compra? Enviar para o último endereço/horário/forma de pagamento)